In [ ]:
!nvidia-smi

Sun Dec 14 00:29:55 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   44C    P8             12W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive/CLIP-VLM/CLIP-VLM"

/content/drive/MyDrive/CLIP-VLM/CLIP-VLM


In [ ]:
# !pip install uv

In [ ]:
!uv pip install -e .

Using Python 3.12.12 environment at: /usr
Resolved 75 packages in 1.12s
Prepared 13 packages in 7.37s
Uninstalled 4 packages in 53ms
Installed 13 packages in 17ms
 + bitsandbytes==0.49.0
 + clip-vlm==0.1.0 (from file:///content/drive/MyDrive/CLIP-VLM/CLIP-VLM)
 + descartes==1.1.0
 + fire==0.7.1
 + ftfy==6.3.1
 - numpy==2.0.2
 + numpy==1.26.4
 + nuscenes-devkit==1.2.0
 + open-clip-torch==3.2.0
 - opencv-python==4.12.0.88
 + opencv-python==4.11.0.86
 - opencv-python-headless==4.12.0.88
 + opencv-python-headless==4.11.0.86
 + parameterized==0.9.0
 + pyquaternion==0.9.9
 - shapely==2.1.2
 + shapely==2.0.7


In [ ]:
# Optional helper: copy dataset from Drive (edit paths or leave disabled).
import os
import subprocess
copy_from_drive = False
drive_dataset = '/content/drive/MyDrive/CLIP-VLM/overlap_front_back_trainval'
local_dataset = '/content/CLIP-VLM/overlap_front_back_trainval'

if copy_from_drive:
    if not os.path.exists(drive_dataset):
        raise FileNotFoundError(drive_dataset)
    if os.path.exists(local_dataset):
        print('Local dataset already exists, skipping copy.')
    else:
        subprocess.run(['cp', '-r', drive_dataset, local_dataset], check=True)
        print('Copied dataset into', local_dataset)
else:
    print('Skipping dataset copy; ensure the folder exists before inference.')


Skipping dataset copy; ensure the folder exists before inference.


In [ ]:
DATASET_DIR = '/content/drive/MyDrive/CLIP-VLM/overlap_front_back_trainval'
TOKEN = '000cf4dfaab54d21a7314036fde74966'
CHECKPOINT = '/content/drive/MyDrive/CLIP-VLM/checkpoints/connector_epoch2.pt'
PROMPT = 'Describe the scene.'
MAX_NEW_TOKENS = 128
QWEN_QUANT = 'bnb-8bit'      # or 'none' on A100
MAX_VIS_TOKENS = 2048
DEVICE = 'cuda'
DISPLAY_IMAGES = True
SAVE_CAPTION_PATH = '/content/drive/MyDrive/CLIP-VLM/preds/token.txt'


In [ ]:
%%bash
python colab_inference.py \
  --checkpoint /content/drive/MyDrive/CLIP-VLM/checkpoints/connector_epoch4.pt \
  --overlap-export-dir /content/drive/MyDrive/CLIP-VLM/overlap_front_back_trainval \
  --token 000cf4dfaab54d21a7314036fde74966 \
  --cameras CAM_FRONT CAM_BACK \
  --prompt "Describe both the front and rear camera views separately, calling out key objects and their positions relative to the vehicle (e.g., “a parked car on the right curb,” “a pedestrian crossing behind”). Focus on road layout, lane markings, nearby vehicles, signage, and any pedestrians or cyclists. Mention differences between the front and back perspectives so the description matches a NuScenes-style reference caption." \
  --max-new-tokens 256 \
  --max-vis-tokens 2048 \
  --device cuda \
  --display \
  --save-caption /content/drive/MyDrive/CLIP-VLM/preds/token.txt


Figure(800x400)

=== Generated Caption ===

 Here is the scene: The front camera view shows a parked car on the right curb, a cyclist riding in the opposite lane, and a pedestrian crossing behind. The rear camera view shows a parked car on the left curb, a cyclist riding in the opposite lane, and a pedestrian crossing behind.
Certainly! Below are detailed descriptions of both the front and rear camera views, focusing on key objects and their positions relative to the vehicle.

### Front Camera View
- **Parked Car**: A parked car is visible on the right curb. It appears to be facing away from the camera, with its rear end towards the viewer.
- **Cyclist**: A cyclist is riding in the opposite lane, positioned slightly ahead of the parked car. The cyclist is moving in the same direction as the car but is further down the road.
- **Pedestrian**: A pedestrian is crossing the street behind the cyclist. The pedestrian is walking in the same direction as the cyclist and the car, but they are f

2025-12-14 00:49:50.040681: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-14 00:49:50.057868: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765673390.079070    8238 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765673390.085443    8238 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765673390.101767    8238 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [ ]:
%%bash
python batch_eval.py \
  --overlap-export-dir /content/drive/MyDrive/CLIP-VLM/overlap_front_back_trainval \
  --output-dir /content/drive/MyDrive/CLIP-VLM/preds \
  --checkpoints-dir /content/drive/MyDrive/CLIP-VLM/checkpoints \
  --prompt "Describe both the front and rear camera views separately, calling out key objects and their positions relative to the vehicle (e.g., “a parked car on the right curb,” “a pedestrian crossing behind”). Focus on road layout, lane markings, nearby vehicles, signage, and any pedestrians or cyclists. Mention differences between the front and back perspectives so the description matches a NuScenes-style reference caption." \
  --max-new-tokens 128 \
  --max-vis-tokens 2048 \
  --device cuda \
  --num-tokens 5


2025-12-14 00:58:51.437070: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-14 00:58:51.455161: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765673931.476971   10606 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765673931.483520   10606 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765673931.500667   10606 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 